# Preparing data

### Imports and Loads

In [9]:
# Necessary imports
import pandas as pd
import numpy as np
import warnings
import gc
import pickle
import os
warnings.filterwarnings("ignore")

# Data load
df_co2 = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")
df_energy = pd.read_csv("https://raw.githubusercontent.com/owid/energy-data/refs/heads/master/owid-energy-data.csv")

### Data merge

In [2]:
common_columns = set(df_co2.columns).intersection(set(df_energy.columns))
print(common_columns)

{'year', 'primary_energy_consumption', 'iso_code', 'energy_per_capita', 'population', 'country', 'gdp', 'energy_per_gdp'}


In [10]:
# G20 countries
g20_countries = [
    'United States', 'China', 'Japan', 'Germany', 
    'United Kingdom', 'France', 'Italy', 'Canada',
    'Brazil', 'Russia', 'India', 'Australia', 
    'Mexico', 'Indonesia', 'Turkey', 'Saudi Arabia',
    'South Africa', 'Argentina', 'South Korea', 'Europe'
]

# Remove iso_code from the common columns
common_cols = list(common_columns)
common_cols.remove('iso_code')

# Merge keys
merge_keys = ['country', 'year']

# Unique columns in each dataframe
remain_common_cols = [col for col in common_cols if col not in merge_keys]
df_co2_unique_cols = list(set(df_co2.columns) - set(df_energy.columns) - set(merge_keys))
df_energy_unique_cols = list(set(df_energy.columns) - set(df_co2.columns) - set(merge_keys))
overlap_cols = list(set(df_co2.columns).intersection(set(df_energy.columns)) - set(common_cols) - set(merge_keys))

# Dataframe with unique rows
df_co2_clean = df_co2.drop_duplicates(merge_keys)
df_energy_clean = df_energy.drop_duplicates(merge_keys)

# Merge common cols and unique cols
col_co2 = remain_common_cols + df_co2_unique_cols
col_energy = df_energy_unique_cols

# Common cols appear once
merged_df = pd.merge(
    df_co2_clean[merge_keys + col_co2], 
    df_energy_clean[merge_keys + col_energy],
    on=merge_keys, 
    how='outer'
)

# Overlapping cols not in common cols
if overlap_cols:
    for col in overlap_cols:
        # Temp cols
        co2_data = df_co2_clean[merge_keys + [col]].rename(columns={col: f"{col}_co2"})
        energy_data = df_energy_clean[merge_keys + [col]].rename(columns={col: f"{col}_energy"})

        merged_df = merged_df.merge(co2_data, on=merge_keys, how='left')
        merged_df = merged_df.merge(energy_data, on=merge_keys, how='left')

        merged_df[col] = merged_df[f"{col}_co2"].combine_first(merged_df[f"{col}_energy"])

        # Drop temp cols
        merged_df = merged_df.drop([f"{col}_co2", f"{col}_energy"], axis=1)

df = merged_df.drop(['iso_code'], axis=1).copy()
g20_df = df[df['country'].isin(g20_countries)].copy()

# Clean up
del df_co2, df_energy, merged_df, df_co2_clean, df_energy_clean
gc.collect()

0

In [11]:
g20_df['country'].unique()

array(['Argentina', 'Australia', 'Brazil', 'Canada', 'China', 'Europe',
       'France', 'Germany', 'India', 'Indonesia', 'Italy', 'Japan',
       'Mexico', 'Russia', 'Saudi Arabia', 'South Africa', 'South Korea',
       'Turkey', 'United Kingdom', 'United States'], dtype=object)

### Time lag

In [12]:
# Time lag feature
def time_lag_feature(df, feature_columns, periods=[1, 2, 3, 4]):
    dup_df = df.copy()

    if 'country' in dup_df.columns and 'year' in dup_df.columns:
        dup_df = dup_df.sort_values(['country', 'year'])

    for country, country_data in dup_df.groupby('country'):
        for col in feature_columns:
            if col in country_data.columns:
                for lag in periods:
                    lag_col_name = f"{col}_lag{lag}"
                    dup_df.loc[country_data.index, lag_col_name] = country_data[col].shift(lag)

    return dup_df

lag_features = ['co2', 'gdp', 'population', 'primary_energy_consumption', 'fossil_fuel_consumption', 'renewables_consumption']

lag_df = g20_df[['country', 'year'] + lag_features].copy()
lag_df = time_lag_feature(lag_df, lag_features, [1, 2, 3, 4])

In [13]:
lag_df.tail(3)

,country,year,co2,gdp,population,primary_energy_consumption,fossil_fuel_consumption,renewables_consumption,co2_lag1,co2_lag2,...,primary_energy_consumption_lag3,primary_energy_consumption_lag4,fossil_fuel_consumption_lag1,fossil_fuel_consumption_lag2,fossil_fuel_consumption_lag3,fossil_fuel_consumption_lag4,renewables_consumption_lag1,renewables_consumption_lag2,renewables_consumption_lag3,renewables_consumption_lag4
51601,United States,2022,5078.871,1.949317e+13,341534041.0,26504.305,21479.428,2993.056,5032.213,4714.628,...,26578.494,26768.986,21170.129,19936.998,21948.143,22212.854,2726.014,2590.245,2475.220,2399.240
51602,United States,2023,4911.391,NaN,343477332.0,26189.199,21102.201,3052.564,5078.871,5032.213,...,24622.646,26578.494,21479.428,21170.129,19936.998,21948.143,2993.056,2726.014,2590.245,2475.220
51603,United States,2024,NaN,NaN,NaN,NaN,NaN,NaN,4911.391,5078.871,...,25956.828,24622.646,21102.201,21479.428,21170.129,19936.998,3052.564,2993.056,2726.014,2590.245


### Simple Analysis of data of g20 countries

In [14]:
# Simple analysis of each features
def df_ini_analysis(df):
    # exclude non-numeric
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'year' in numeric_cols:
        numeric_cols.remove('year')
    
    stats = []

    for col in numeric_cols:
        total_values = len(df)
        non_null_values = df[col].notna().sum()
        coverage = (non_null_values / total_values) * 100

        if non_null_values > 0:
            year_range = f"{df[df[col].notna()]['year'].min()} - {df[df[col].notna()]['year'].max()}"
            countries_with_data = df[df[col].notna()]['country'].nunique()
        else:
            year_range = "Not available"
            countries_with_data = 0

        stats.append({
            'Feature': col,
            'Coverage_(%)': coverage,
            'Non_null_values': non_null_values,
            'Year_range': year_range,
            'Countries_with_data': countries_with_data
        })
        
    avail_df = pd.DataFrame(stats)
    avail_df = avail_df.sort_values('Coverage_(%)', ascending=False)

    print("Top 70 features by coverage")
    print(avail_df.head(70).to_string(index=False))

    return avail_df

In [15]:
df_ini_analysis(g20_df)

Top 70 features by coverage
                                  Feature  Coverage_(%)  Non_null_values  Year_range  Countries_with_data
                               population     95.717131             3844 1750 - 2023                   20
                                      co2     90.587649             3638 1750 - 2023                   20
                           cumulative_co2     90.587649             3638 1750 - 2023                   20
              share_global_cumulative_co2     90.587649             3638 1750 - 2023                   20
                         share_global_co2     90.587649             3638 1750 - 2023                   20
                           co2_growth_abs     89.890438             3610 1751 - 2023                   20
                          co2_growth_prct     88.471116             3553 1751 - 2023                   20
                    share_global_coal_co2     88.446215             3552 1750 - 2023                   20
                  

,Feature,Coverage_(%),Non_null_values,Year_range,Countries_with_data
2,population,95.717131,3844,1750 - 2023,20
36,co2,90.587649,3638,1750 - 2023,20
38,cumulative_co2,90.587649,3638,1750 - 2023,20
59,share_global_cumulative_co2,90.587649,3638,1750 - 2023,20
75,share_global_co2,90.587649,3638,1750 - 2023,20
...,...,...,...,...,...
40,share_global_other_co2,11.852590,476,1990 - 2023,14
179,biofuel_cons_change_pct,10.956175,440,1971 - 2023,19
120,other_renewable_exc_biofuel_electricity,6.050797,243,1990 - 2024,8
142,other_renewables_elec_per_capita_exc_biofuel,6.050797,243,1990 - 2024,8


### Save Dataframes and analysis

In [34]:
# Create markdown README
def create_readme(analysis_df, save_dir):

    readme_content = f"""# G20 Countries CO2, GDP, and Energy Simple Data Analysis
    
### Feature Analysis sorted by Coverage and over 60%

| Feature | Coverage (%) | Non-null Values | Year Range | Countries with Data |
|---------|--------------|-----------------|------------|---------------------|"""


    # Features with greater than 60% of coverage
    acpt_coverage = analysis_df[analysis_df['Coverage_(%)'] >= 60.0]
    for _, row in acpt_coverage.iterrows():
        readme_content += f"\n| {row['Feature']} | {row['Coverage_(%)']:.1f} | {row['Non_null_values']} | {row['Year_range']} | {row['Countries_with_data']} |"

    # Save README
    readme_path = os.path.join(save_dir, f'feature_g20_analysis.md')
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme_content)

In [37]:
# Saving dataframes and analysis
def save_dfs_analysis(g20_df, lag_df, save_dir='data_export'):

    # Create dir
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Directory created: {save_dir}")
    
    # Save dataframes as pickle
    print(f"Saving dfs as pickle")
    g20_df.to_pickle(os.path.join(save_dir, f"g20_df.pkl"))
    lag_df.to_pickle(os.path.join(save_dir, f"lag_df.pkl"))

    # Save dataframes as csv
    print(f"Saving dfs as csv")
    g20_df.to_csv(os.path.join(save_dir, f"g20_df.csv"), index=False)
    lag_df.to_csv(os.path.join(save_dir, f"lag_df.csv"), index=False)

    # Save analysis and as markdown README
    analysis_df = df_ini_analysis(g20_df)
    #analysis_df.to_csv(os.path.join(save_dir, f'feature_g20_analysis.csv'), index=False)

    create_readme(analysis_df, save_dir)

In [38]:
save_dfs_analysis(g20_df, lag_df)

Saving dfs as pickle
Saving dfs as csv
Top 70 features by coverage
                                  Feature  Coverage_(%)  Non_null_values  Year_range  Countries_with_data
                               population     95.717131             3844 1750 - 2023                   20
                                      co2     90.587649             3638 1750 - 2023                   20
                           cumulative_co2     90.587649             3638 1750 - 2023                   20
              share_global_cumulative_co2     90.587649             3638 1750 - 2023                   20
                         share_global_co2     90.587649             3638 1750 - 2023                   20
                           co2_growth_abs     89.890438             3610 1751 - 2023                   20
                          co2_growth_prct     88.471116             3553 1751 - 2023                   20
                    share_global_coal_co2     88.446215             3552 1750 - 2023 